# SageMakerのserializerとdeserializerを理解する

Serializerは、推論エンドポイントに対して、シリアライズされたデータと、CONTENT_TYPEを提供します。

このノートブックでは、推論エンドポイントにNumpyArrayとCSV形式でのリクエストを想定して、
NumpyArrayデータとCSVデータがどのようにシリアライズされるか、どのようにデシリアライズされるかを学びます。

SageMakerのシリアライズ/デシリアライズの仕組みを知ることで、より推論の仕組みを深く理解します。

ノートブックは20分程度で実行できます。

## SageMakerの仕組み（仮説）

* SageMakerのコントロールプレーン（サーバ）がある。
    * それは、pingを打って各推論エンドポイントが動いてるかヘルスチェックして、把握している。
    
* 推論エンドポイントへは、.predict()(SageMakerSDKの場合) or invoke_endpoint()(boto3の場合）でデータを投げる
    * predictも結局はinvoke_endpoint()している

https://github.com/aws/sagemaker-python-sdk/blob/885423c26ce7288283bbca7d9c1c53c4d0ccf103/src/sagemaker/predictor.py#L123


invoke_endpoint()すると、SageMakerに推論先(同じエンドポイントでも、variantごとにインスタンスタイプを持っているので、別IPアドレスのはず。同じvariantでも複数のインスタンスを持ち、それらも別IPのはず）を聞きに行き、返された宛先のエンドポイントにデータを投げていると予想。
* endpointやvariantを指定しているので、SageMakerに場所を聞く必要があると予想。SageMakerはDNSのような役割をする。
    * これにより、variantsへのロードバランスをSageMakerが行える。（AutoScaleはSageMakerではなく、他の機構が行なっているはず）
* SageMakerから帰ってきた宛先に/invocationを投げる。推論エンドポイントは/invocationに返答する。

invoke_endpoint()の前の、Predictorクラス作成の時に、SerializerとDeserializerを指定している。
つまり、データ投げる前のクライアント側でシリアライズして、エンドポイントに投げる。
エンドポイントからの応答（推論結果）は、シリアルデータでクライアントに返ってくる。
デシリアライズをクライアント側で実施する。


シリアライザ、デシリアライザは、Estimatorクラスのdeploy()の時に設定します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

Modelクラスによる既存のモデル読み込みからのdeploy()

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy



# 1. Serializerの理解
シリアライズはクライアント側で実行され、シリアライズされたデータは推論エンドポイントにinvokeされます。

Serializerのコードは、GitHubで確認することができます。

GitHubのソースコード

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/serializers.py


input形式は様々

outputはシリアライザで指定したクラスによる

## 1-1. データ準備

In [1]:
import numpy as np

In [2]:
# ndarray型の推論エンドポイントへのインプットデータ
data_ndarr = np.array([[0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81],
                       [0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05],
                       [4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66]])

# string型の推論エンドポイントへのインプットデータ
data_str = '0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'


In [3]:
# 確認
print(type(data_ndarr))
print('=' * 50)
print(data_ndarr)
print('=' * 50)
data_ndarr

<class 'numpy.ndarray'>
[[2.53870e-01 0.00000e+00 6.91000e+00 0.00000e+00 4.48000e-01 5.39900e+00
  9.53000e+01 5.87000e+00 3.00000e+00 2.33000e+02 1.79000e+01 3.96900e+02
  3.08100e+01]
 [1.95100e-02 1.75000e+01 1.38000e+00 0.00000e+00 4.16100e-01 7.10400e+00
  5.95000e+01 9.22290e+00 3.00000e+00 2.16000e+02 1.86000e+01 3.93240e+02
  8.05000e+00]
 [4.64689e+00 0.00000e+00 1.81000e+01 0.00000e+00 6.14000e-01 6.98000e+00
  6.76000e+01 2.53290e+00 2.40000e+01 6.66000e+02 2.02000e+01 3.74680e+02
  1.16600e+01]]


array([[2.53870e-01, 0.00000e+00, 6.91000e+00, 0.00000e+00, 4.48000e-01,
        5.39900e+00, 9.53000e+01, 5.87000e+00, 3.00000e+00, 2.33000e+02,
        1.79000e+01, 3.96900e+02, 3.08100e+01],
       [1.95100e-02, 1.75000e+01, 1.38000e+00, 0.00000e+00, 4.16100e-01,
        7.10400e+00, 5.95000e+01, 9.22290e+00, 3.00000e+00, 2.16000e+02,
        1.86000e+01, 3.93240e+02, 8.05000e+00],
       [4.64689e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.14000e-01,
        6.98000e+00, 6.76000e+01, 2.53290e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.74680e+02, 1.16600e+01]])

In [5]:
# 確認
print(type(data_str))
print('=' * 50)
print(data_str)
print('=' * 50)
data_str

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

In [8]:
# 推論実行で使っていたコード
#with open(local_test, 'r') as f:
#    payload = f.read().strip()
#    print(type(payload))
#    print(payload)
#print('=' * 20)
#payload

いろいろなシリアライザ

https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html


## 1.2 ndarray型のデータをシリアライザに適用する

In [12]:
from sagemaker.serializers import CSVSerializer
from sagemaker.serializers import NumpySerializer

In [14]:
print(CSVSerializer().content_type)
print(NumpySerializer().content_type)

text/csv
application/x-npy


### 1-2-1. CSVSerializerでシリアライズする

In [15]:
serialized = CSVSerializer().serialize(data_ndarr)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

### 1-2-2. NumpySerializerでシリアライズする

In [18]:
serialized = NumpySerializer().serialize(data_ndarr)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'bytes'>
b'\x93NUMPY\x01\x00v\x00{\'descr\': \'<f8\', \'fortran_order\': False, \'shape\': (3, 13), }                                                         \n\x8f\xdf\xdb\xf4g?\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\xa4p=\n\xd7\xa3\x1b@\x00\x00\x00\x00\x00\x00\x00\x00y\xe9&1\x08\xac\xdc?\x7fj\xbct\x93\x98\x15@33333\xd3W@{\x14\xaeG\xe1z\x17@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00 m@fffff\xe61@fffff\xcex@\x8f\xc2\xf5(\\\xcf>@\x98\xa3\xc7\xefm\xfa\x93?\x00\x00\x00\x00\x00\x801@\x14\xaeG\xe1z\x14\xf6?\x00\x00\x00\x00\x00\x00\x00\x00\xfee\xf7\xe4a\xa1\xda?\xd1"\xdb\xf9~j\x1c@\x00\x00\x00\x00\x00\xc0M@\x8a\x8e\xe4\xf2\x1fr"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00k@\x9a\x99\x99\x99\x99\x992@\xa4p=\n\xd7\x93x@\x9a\x99\x99\x99\x99\x19 @\x11p\x08Uj\x96\x12@\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x99\x99\x99\x99\x192@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xa5\xe3?\xecQ\xb8\x1e\x85\xeb\x1b@fffff\xe6P@\xa5N@\x13aC\x04@\x00\x00\x00\x00\x00\x008@\x00\x00\x00\x00

b'\x93NUMPY\x01\x00v\x00{\'descr\': \'<f8\', \'fortran_order\': False, \'shape\': (3, 13), }                                                         \n\x8f\xdf\xdb\xf4g?\xd0?\x00\x00\x00\x00\x00\x00\x00\x00\xa4p=\n\xd7\xa3\x1b@\x00\x00\x00\x00\x00\x00\x00\x00y\xe9&1\x08\xac\xdc?\x7fj\xbct\x93\x98\x15@33333\xd3W@{\x14\xaeG\xe1z\x17@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00 m@fffff\xe61@fffff\xcex@\x8f\xc2\xf5(\\\xcf>@\x98\xa3\xc7\xefm\xfa\x93?\x00\x00\x00\x00\x00\x801@\x14\xaeG\xe1z\x14\xf6?\x00\x00\x00\x00\x00\x00\x00\x00\xfee\xf7\xe4a\xa1\xda?\xd1"\xdb\xf9~j\x1c@\x00\x00\x00\x00\x00\xc0M@\x8a\x8e\xe4\xf2\x1fr"@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00k@\x9a\x99\x99\x99\x99\x992@\xa4p=\n\xd7\x93x@\x9a\x99\x99\x99\x99\x19 @\x11p\x08Uj\x96\x12@\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x99\x99\x99\x99\x192@\x00\x00\x00\x00\x00\x00\x00\x00\xd9\xce\xf7S\xe3\xa5\xe3?\xecQ\xb8\x1e\x85\xeb\x1b@fffff\xe6P@\xa5N@\x13aC\x04@\x00\x00\x00\x00\x00\x008@\x00\x00\x00\x00\x00\xd0\x84@333

## 1.3 string型のデータをシリアライザに適用する

### 1-3-1. CSVSerializerでシリアライズする

In [19]:
serialized = CSVSerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

### 1-3-2. NumpySerializerでシリアライズする

In [20]:
serialized = NumpySerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

<class 'bytes'>
b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00

b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x000\x00\x00\x0

SageMakerの動き
* クライアント側でデータがシリアライズされる。シリアルデータを推論エンドポイントに送る。
* ====== SageMaker 内部 ==========
* 推論エンドポイントは、シリアル化されたデータを受け取る
* SageMakerのコードで、デシリアライズする。
* input_fn実行
* predict_fn実行
* output_fn実行
* データをシリアライズする。
* クライアントに送信
* ====== SageMaker 内部 ==========
* クライアント側で、デシリアライズする。

In [ ]:
### str型のCSVフォーマットをJSONフォーマットにシリアライズする場合
from sagemaker.serializers import JSONSerializer
serialized = JSONSerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

In [ ]:
### str型のCSVフォーマットをシリアライズする場合
from sagemaker.serializers import CSVSerializer
serialized = CSVSerializer().serialize(data_str)

# 確認
print(type(serialized))
print('=' * 50)
print(serialized)
print('=' * 50)
serialized

# 2.デシリアライズの確認
クライアントは、推論エンドポイントからシリアルデータを受け取りますので、それをクライアント側でデシリアライズします。

デシリアライザは推論エンドポイントからの返答（シリアルデータ）をデシリアライズすることと、ACCEPTが設定されており、推論エンドポイントからどの形式でデータを受領するかを定義できます。これは、output_fn()での出力と合わせる必要があります。


GitHubのソースコード

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/deserializers.py


inputは、推論エンドポイントから受信したシリアルデータ。形式はJSONやndarray

outputは、クラスで指定した形式（JSON, ndarray, pandasなど）


LightGBMは推論結果をndarray型で出力するので、ndarray型をシリアライズして、クライアントに渡すことを想定する。

In [ ]:
from sagemaker.deserializers import PandasDeserializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import NumpyDeserializer

In [ ]:
print(PandasDeserializer().ACCEPT)
print(JSONDeserializer().ACCEPT)
print(NumpyDeserializer().ACCEPT)

In [ ]:
import botocore
import json
from io import BytesIO

In [ ]:
# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))

In [ ]:
PandasDeserializer().ACCEPT

In [ ]:
deserialized = PandasDeserializer().deserialize(body, 'text/csv')

In [ ]:
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

In [ ]:
# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))

In [ ]:
deserialized2 = PandasDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット

In [ ]:
print(type(deserialized2))
print('='*30)
print(deserialized2)
print('='*30)
deserialized2

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

In [ ]:
body_ndarr = np.array([
                       19.95642073217597,
                       27.844891841022335,
                       23.747437427003455
                      ])

In [ ]:
# エンコード。(encode()はデフォルトでutf-8。)
#body_encoded = json.dumps(body_json).encode()
#body_encoded2 = body_nparr.tobytes()
body_encoded2 = body_ndarr.dumps()

# StreamingBodyへ整形する。
#body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))
body2 = botocore.response.StreamingBody(BytesIO(body_encoded2),len(body_encoded2))

#deserialized = NumpyDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット
deserialized = NumpyDeserializer().deserialize(body2, 'application/x-npy') ### JSONがdeserializerのインプット

In [ ]:
print(body_nparr)
print(deserialized)
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()
body_encoded2 = body_nparr.tobytes()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))
body2 = botocore.response.StreamingBody(BytesIO(body_encoded2),len(body_encoded2))

#deserialized = NumpyDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット
deserialized = NumpyDeserializer().deserialize(body2, 'application/x-npy') ### JSONがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

In [ ]:
print(type(body_encoded))
print(body_encoded)

print(type(body))
print(body)


print(type(body_nparr))
print(body_nparr)

print(type(body_encoded2))
print(body_encoded2)

print(type(body2))
print(body2)

# 参考

botocore.response

https://botocore.amazonaws.com/v1/documentation/api/latest/reference/response.html

raw_streamを入力する必要がある。


バイナリ I/O
https://docs.python.org/ja/3/library/io.html#binary-i-o


BytesIO はインメモリーのバイナリストリームです:

f = io.BytesIO(b"some initial binary data: \x00\x01")

In [ ]:
from sagemaker.deserializers import NumpyDeserializer


# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(b'{"hogehoge":1}'),len(b'{"hogehoge":1}'))

deserialized = NumpyDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット
#deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy') ### JSONがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(b'{"hogehoge":1}'),len(b'{"hogehoge":1}'))

deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy') ### ndarrayがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

In [ ]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [ ]:
# np.load(io.BytesIO(stream.read()), allow_pickle=self.allow_pickle)　が動かないとエラー

np.load(BytesIO(b'[1,1,1]'), allow_pickle='allow_pickle') # np.load()でエラー発生

In [ ]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [ ]:
body_nparr

In [ ]:
np.save('hoge', body_nparr)

In [ ]:
body = botocore.response.StreamingBody(b'{"hogehoge":1}',len(b'{"hogehoge":1}'))

In [ ]:
body.read()

In [ ]:
body.seek()

In [ ]:
BytesIO(b'{"hogehoge":1}')


nupy.load()

https://numpy.org/doc/stable/reference/generated/numpy.load.html

The file to read. File-like objects must support the seek() and read() methods and must always be opened in binary mode. 

In [ ]:
BytesIO(b'[1,1,1]').seek(50000)

In [ ]:
BytesIO(b'[1,1,1]').read(10000)

BytesIOはseekもreadもできる。

In [ ]:
np.load(BytesIO(b' a'), allow_pickle=True) # np.load()でエラー発生

b'aaaa'のバイト列がいけてないのか？pickleであることを示す文字列がない？で、seek()で失敗している？？


中身が想定しているものではないのかも
https://teratail.com/questions/302899

ファイルがおかしい場合にエラーとなっている事例のようだ


In [ ]:
np.load(BytesIO(body_nparr.dumps()), allow_pickle=True) # np.load()でエラー発生

# 解答
numpyのndarrayを、ファイルではなく、pickle文字列に変換する必要がある。
そのために、numpy.ndarray.dumps()を使う

https://numpy.org/doc/stable/reference/generated/numpy.ndarray.dumps.html

In [ ]:
np.load(BytesIO(body_nparr.dumps()), allow_pickle=True) # np.load()でエラー発生

### こうすることで、doby_nparrがpickleのstringに変換され、BytesIO()によってseek()もread()もできるストリーム（file_alike)に変換される。
### np.load()でこれを読み込むことができる。

In [ ]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [ ]:
body_nparr.tobytes()